In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1140977,2021-04-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1140978,2021-04-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1140979,2021-04-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1140980,2021-04-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30663,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30665,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30667,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30669,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30671,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
703956,2021-04-01,Arkansas,Arkansas,5001,2077,36.00,5000,Arkansas,AR,Arkansas,State,3017804
703958,2021-04-02,Arkansas,Arkansas,5001,2077,36.00,5000,Arkansas,AR,Arkansas,State,3017804
703960,2021-04-03,Arkansas,Arkansas,5001,2079,36.00,5000,Arkansas,AR,Arkansas,State,3017804
703962,2021-04-04,Arkansas,Arkansas,5001,2079,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1140977,2021-04-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1140978,2021-04-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1140979,2021-04-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1140980,2021-04-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-05') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
440,2021-04-05,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48
878,2021-04-05,Cook,Illinois,504203,10263.00,17031,IL,County,5150233,9789.91,199.27
1315,2021-04-05,Orange,California,267001,4772.00,6059,CA,County,3175692,8407.65,150.27
1751,2021-04-05,Maricopa,Arizona,526070,9668.00,4013,AZ,County,4485414,11728.46,215.54
2187,2021-04-05,Los Angeles,California,1223104,23301.00,6037,CA,County,10039107,12183.39,232.10
...,...,...,...,...,...,...,...,...,...,...,...
1140405,2021-04-05,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1140580,2021-04-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1140724,2021-04-05,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1140864,2021-04-05,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
440,2021-04-05,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48,68.48,4047.16
878,2021-04-05,Cook,Illinois,504203,10263.00,17031,IL,County,5150233,9789.91,199.27,199.27,9789.91
1315,2021-04-05,Orange,California,267001,4772.00,6059,CA,County,3175692,8407.65,150.27,150.27,8407.65
1751,2021-04-05,Maricopa,Arizona,526070,9668.00,4013,AZ,County,4485414,11728.46,215.54,215.54,11728.46
2187,2021-04-05,Los Angeles,California,1223104,23301.00,6037,CA,County,10039107,12183.39,232.10,232.10,12183.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140405,2021-04-05,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1140580,2021-04-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1140724,2021-04-05,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1140864,2021-04-05,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
736039,2021-04-05,Hale,Alabama,2173,73.00,1065,AL,County,14651,14831.75,498.26,498.26,14831.75,1
801059,2021-04-05,Clarke,Alabama,3454,60.00,1025,AL,County,23622,14621.96,254.00,254.00,14621.96,2
576081,2021-04-05,Lowndes,Alabama,1363,53.00,1085,AL,County,9726,14013.98,544.93,544.93,14013.98,3
428766,2021-04-05,Franklin,Alabama,4189,82.00,1059,AL,County,31362,13356.93,261.46,261.46,13356.93,4
510866,2021-04-05,Etowah,Alabama,13658,345.00,1055,AL,County,102268,13355.11,337.35,337.35,13355.11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082859,2021-04-05,Platte,Wyoming,611,11.00,56031,WY,County,8393,7279.88,131.06,131.06,7279.88,19
702471,2021-04-05,Converse,Wyoming,984,17.00,56009,WY,County,13822,7119.09,122.99,122.99,7119.09,20
873016,2021-04-05,Lincoln,Wyoming,1361,12.00,56023,WY,County,19830,6863.34,60.51,60.51,6863.34,21
971382,2021-04-05,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
576081,2021-04-05,Lowndes,Alabama,1363,53.00,1085,AL,County,9726,14013.98,544.93,544.93,14013.98,3,1
736039,2021-04-05,Hale,Alabama,2173,73.00,1065,AL,County,14651,14831.75,498.26,498.26,14831.75,1,2
245225,2021-04-05,Walker,Alabama,7073,274.00,1127,AL,County,63521,11134.90,431.35,431.35,11134.90,28,3
621198,2021-04-05,Greene,Alabama,906,34.00,1063,AL,County,8111,11170.02,419.18,419.18,11170.02,27,4
575329,2021-04-05,Crenshaw,Alabama,1501,57.00,1041,AL,County,13772,10898.93,413.88,413.88,10898.93,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619697,2021-04-05,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
873016,2021-04-05,Lincoln,Wyoming,1361,12.00,56023,WY,County,19830,6863.34,60.51,60.51,6863.34,21,20
851304,2021-04-05,Uinta,Wyoming,2134,12.00,56041,WY,County,20226,10550.78,59.33,59.33,10550.78,4,21
244076,2021-04-05,Teton,Wyoming,3632,9.00,56039,WY,County,23464,15479.03,38.36,38.36,15479.03,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,16,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,16,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,16,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,16,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,16,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138296,2021-04-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1138297,2021-04-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1138298,2021-04-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1138299,2021-04-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
732987,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14831.75,2,1,1.00
732988,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14831.75,2,1,0.00
732989,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14831.75,2,1,0.00
732990,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14831.75,2,1,0.00
732991,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14831.75,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887580,2021-04-01,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
887581,2021-04-02,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
887582,2021-04-03,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
887583,2021-04-04,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
573773,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,1.00,0.00
573774,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
573775,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
573776,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
573777,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571888,2021-04-01,Albany,Wyoming,3979,11.00,56001,WY,County,38880,10234.05,28.29,28.29,10262.35,23,5,9.00,0.00
571889,2021-04-02,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10262.35,23,5,5.00,0.00
571890,2021-04-03,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10262.35,23,5,0.00,0.00
571891,2021-04-04,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10262.35,23,5,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-05') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
292685,2021-04-05,Imperial,California,27588,711.00,6025,CA,County,181215,15223.91,392.35,392.35,15223.91,1,2,16.00,0.00
2187,2021-04-05,Los Angeles,California,1223104,23301.00,6037,CA,County,10039107,12183.39,232.10,232.10,12183.39,2,5,333.00,0.00
145837,2021-04-05,San Bernardino,California,291727,4423.00,6071,CA,County,2180085,13381.45,202.88,202.88,13381.45,3,4,79.00,1.00
512326,2021-04-05,Inyo,California,1377,36.00,6027,CA,County,18039,7633.46,199.57,199.57,7633.46,4,25,1.00,0.00
65261,2021-04-05,Stanislaus,California,59460,1004.00,6099,CA,County,550660,10797.95,182.33,182.33,10797.95,5,9,118.00,1.00
30267,2021-04-05,Riverside,California,295426,4389.00,6065,CA,County,2470546,11957.92,177.65,177.65,11957.92,6,6,690.00,27.00
80898,2021-04-05,Tulare,California,49129,818.00,6107,CA,County,466195,10538.29,175.46,175.46,10538.29,7,10,29.00,0.00
51935,2021-04-05,San Joaquin,California,70245,1313.00,6077,CA,County,762148,9216.71,172.28,172.28,9216.71,8,19,128.00,3.00
29477,2021-04-05,Fresno,California,99702,1622.00,6019,CA,County,999101,9979.17,162.35,162.35,9979.17,9,14,100.00,0.00
464337,2021-04-05,Merced,California,30906,442.00,6047,CA,County,277680,11130.08,159.18,159.18,11130.08,10,8,219.00,3.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1082410,2021-04-05,Lassen,California,5656,1.00,6035,CA,County,30573,18499.98,3.27,3.27,18499.98,56,1,4.00,0.00
292685,2021-04-05,Imperial,California,27588,711.00,6025,CA,County,181215,15223.91,392.35,392.35,15223.91,1,2,16.00,0.00
621140,2021-04-05,Kings,California,22669,241.00,6031,CA,County,152940,14822.15,157.58,157.58,14822.15,11,3,0.00,0.00
145837,2021-04-05,San Bernardino,California,291727,4423.00,6071,CA,County,2180085,13381.45,202.88,202.88,13381.45,3,4,79.00,1.00
2187,2021-04-05,Los Angeles,California,1223104,23301.00,6037,CA,County,10039107,12183.39,232.10,232.10,12183.39,2,5,333.00,0.00
30267,2021-04-05,Riverside,California,295426,4389.00,6065,CA,County,2470546,11957.92,177.65,177.65,11957.92,6,6,690.00,27.00
183321,2021-04-05,Kern,California,106480,1291.00,6029,CA,County,900202,11828.46,143.41,143.41,11828.46,14,7,20.00,5.00
464337,2021-04-05,Merced,California,30906,442.00,6047,CA,County,277680,11130.08,159.18,159.18,11130.08,10,8,219.00,3.00
65261,2021-04-05,Stanislaus,California,59460,1004.00,6099,CA,County,550660,10797.95,182.33,182.33,10797.95,5,9,118.00,1.00
80898,2021-04-05,Tulare,California,49129,818.00,6107,CA,County,466195,10538.29,175.46,175.46,10538.29,7,10,29.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
292685,2021-04-05,Imperial,California,27588,711.00,6025,CA,County,181215,15223.91,392.35,392.35,15223.91,1,2,16.00,0.00
2187,2021-04-05,Los Angeles,California,1223104,23301.00,6037,CA,County,10039107,12183.39,232.10,232.10,12183.39,2,5,333.00,0.00
145837,2021-04-05,San Bernardino,California,291727,4423.00,6071,CA,County,2180085,13381.45,202.88,202.88,13381.45,3,4,79.00,1.00
512326,2021-04-05,Inyo,California,1377,36.00,6027,CA,County,18039,7633.46,199.57,199.57,7633.46,4,25,1.00,0.00
65261,2021-04-05,Stanislaus,California,59460,1004.00,6099,CA,County,550660,10797.95,182.33,182.33,10797.95,5,9,118.00,1.00
30267,2021-04-05,Riverside,California,295426,4389.00,6065,CA,County,2470546,11957.92,177.65,177.65,11957.92,6,6,690.00,27.00
80898,2021-04-05,Tulare,California,49129,818.00,6107,CA,County,466195,10538.29,175.46,175.46,10538.29,7,10,29.00,0.00
51935,2021-04-05,San Joaquin,California,70245,1313.00,6077,CA,County,762148,9216.71,172.28,172.28,9216.71,8,19,128.00,3.00
29477,2021-04-05,Fresno,California,99702,1622.00,6019,CA,County,999101,9979.17,162.35,162.35,9979.17,9,14,100.00,0.00
464337,2021-04-05,Merced,California,30906,442.00,6047,CA,County,277680,11130.08,159.18,159.18,11130.08,10,8,219.00,3.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15223.91,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3832,04/01/21,Lassen,5652,1.00,18486.90,3.27,3.27,18499.98,56,1,0.00,0.00
3833,04/02/21,Lassen,5652,1.00,18486.90,3.27,3.27,18499.98,56,1,0.00,0.00
3834,04/03/21,Lassen,5652,1.00,18486.90,3.27,3.27,18499.98,56,1,0.00,0.00
3835,04/04/21,Lassen,5652,1.00,18486.90,3.27,3.27,18499.98,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15223.91,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15223.91,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3918,04/01/21,Merced,30649,438.00,11037.53,157.74,159.18,11130.08,10,8,56.00,2.00
3919,04/02/21,Merced,30687,439.00,11051.21,158.10,159.18,11130.08,10,8,38.00,1.00
3920,04/03/21,Merced,30687,439.00,11051.21,158.10,159.18,11130.08,10,8,0.00,0.00
3921,04/04/21,Merced,30687,439.00,11051.21,158.10,159.18,11130.08,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)